# Prepare Data From CSV Recording

## Load File

Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.

The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.

The columns are reorded in the order of head/r_controller/l_controller/waist/r_foot/l_foot.

The new trimmed file is written to a directory (/test_data or /train_data), for further manipulation and loading into the model.


In [34]:
import pandas as pd


#Read in CSV
def GetRecording(path):
    recording_path = "../recordings/"
    file_name = input("Input Recording File Name")
    try:
        dataframe = pd.read_csv(recording_path + file_name + ".csv")
        return dataframe, file_name
    except: 
        print("Error Reading File: Check Spelling and Try Again")
        return 0
    
    
#Seperate each tracker to seperate dataframe

def GetColByName(dataframe):
    HMD = dataframe.loc[:, ["HMD0_tx", "HMD0_ty", "HMD0_tz"]]
    
    controller_1 = dataframe.loc[:, ['controller3_tx', 'controller3_ty', 'controller3_tz']]

    controller_2 = dataframe.loc[:, ['controller4_tx', 'controller4_ty', 'controller4_tz']]

    tracker_1 = dataframe.loc[:, ['generic7_tx', 'generic7_ty', 'generic7_tz']]

    tracker_2 = dataframe.loc[:, ['generic8_tx', 'generic8_ty', 'generic8_tz']]

    tracker_3 = dataframe.loc[:, ['generic9_tx', 'generic9_ty', 'generic9_tz']]

    joined = pd.concat([HMD,controller_1, controller_2, tracker_1 ,tracker_2 ,tracker_3], axis=1)
    return joined

def AssignTracker(dataframe):
    display(dataframe.iloc[0:1,:])
    trackerNum = 7
    for x in range(3):
        trackerStr = str(trackerNum)
        tracker = input('assign generic' + trackerStr)
        dataframe.rename(columns={'generic' + trackerStr + '_tx': tracker + '_x', 'generic' + trackerStr + '_ty': tracker + "_y", 'generic' + trackerStr + '_tz': tracker + '_z'}, inplace=True)
        trackerNum += 1
        
    controllerNum = 3
    for x in range(2):
        controllerStr = str(controllerNum)
        controller = input('assign controller' + controllerStr)
        dataframe.rename(columns={'controller' + controllerStr + '_tx': controller + '_x', 'controller' + controllerStr + '_ty': controller + "_y", 'controller' + controllerStr + '_tz': controller + '_z'}, inplace=True)
        controllerNum += 1
    dataframe.rename(columns={'HMD0_tx': 'head_x', 'HMD0_ty': 'head_y', 'HMD0_tz': 'head_z'}, inplace=True)
    return dataframe

def GetDirectory():
    choice = input("train or test data:")
    if choice == "test":
        output_path = "../test_data/"
    else:
        output_path = "../train_data/"
    return output_path

def OrderFeatures(dataframe):
    head = dataframe.loc[:, ['head_x', 'head_y', 'head_z']]
    l_controller = dataframe.loc[:, ['l_controller_x', 'l_controller_y', 'l_controller_z']]
    r_controller = dataframe.loc[:, ['r_controller_x', 'r_controller_y', 'r_controller_z']]
    waist = dataframe.loc[:, ['waist_x', 'waist_y', 'waist_z']]
    r_foot = dataframe.loc[:, ['r_foot_x', 'r_foot_y', 'r_foot_z']]
    l_foot = dataframe.loc[:, ['l_foot_x', 'l_foot_y', 'l_foot_z']]
    reordered = pd.concat([head , r_controller, l_controller, waist, r_foot, l_foot], axis=1)
    return reordered

    
    
def WriteOutput(path, dataframe, filename):
    output_file = path + filename + "_trimmed.csv"
    dataframe.to_csv(output_file, index = False)
    print(file_name + " output to " + path)
    
    

In [40]:
recording_path = " ../recordings"

dataframe, file_name = GetRecording(recording_path)
joined = GetColByName(dataframe)
renamed = AssignTracker(joined)
path = GetDirectory()
reordered = OrderFeatures(renamed)
WriteOutput(path, reordered, file_name)


Input Recording File Name arm_raise_2


,HMD0_tx,HMD0_ty,HMD0_tz,controller3_tx,controller3_ty,controller3_tz,controller4_tx,controller4_ty,controller4_tz,generic7_tx,generic7_ty,generic7_tz,generic8_tx,generic8_ty,generic8_tz,generic9_tx,generic9_ty,generic9_tz
0,7.488835,160.079437,-6.116605,-16.509771,80.914047,-10.186529,29.804235,82.701195,-13.233584,29.640865,11.117661,-0.257885,6.043923,102.261971,-9.864593,-10.53499,10.7512,0.808924


assign generic7 r_foot
assign generic8 waist
assign generic9 l_foot
assign controller3 l_controller
assign controller4 r_controller
train or test data: test


arm_raise_2 output to ../test_data/


# Data Normalization

## Data Scaling

The new CSV is loaded into memory, chosen through an input prompt
The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).
These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"


#read in formatted CSV
def ReadCSV(path):
    file_name = input("Input File Name")
    try:
        dataframe = pd.read_csv(path + file_name + ".csv")
        print("Dataframe created")
    except:
        print("Error Reading File")
    return dataframe

def SplitFeaturesLabels(dataframe):
    x = dataframe.iloc[:, 0:9]
    y = dataframe.iloc[:, 9:18]
    return x, y

#Load data into Numpy array
def LoadArray(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    return x_array, y_array


def NormalizeValues (x, y):
    x =  np.divide(x, 100)
    y =  np.divide(y, 100)
    return x, y

def SampleSize(x, y):
    x_samples = x[0:600,:]
    y_samples = y[0:600,:]
    return x_samples, y_samples

def RoundValues(x, y): 
    x_rounded = np.around(x, 3)
    y_rounded = np.around(y, 3)
    return x_rounded, y_rounded



    
    

    





# Create Training Data

In [93]:
train_path ="../train_data/"
#load train data from csv
train_dataframe = ReadCSV(train_path)

#split features and labels into seperate dataframes
x_train_df, y_train_df = SplitFeaturesLabels(train_dataframe)

#convert features and labels to numpy array
x_train, y_train = LoadArray(x_train_df, y_train_df)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train, y_train)


x_samples, y_samples = SampleSize(x_train_normalized, y_train_normalized)

print(x_samples.shape, x_samples)
print(y_samples.shape, y_samples)
print(x_train_df)
#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)





Input File Name walking_5_trimmed


Dataframe created
(600, 9) [[ 0.10940075  1.62021332 -0.00498736 ... -0.14921427  0.82919815
   0.02731556]
 [ 0.1088817   1.61971085 -0.0054888  ... -0.15115108  0.82919815
   0.02825213]
 [ 0.10831941  1.61920166 -0.00607767 ... -0.15327472  0.82896408
   0.02916744]
 ...
 [-0.54247967  1.73348389  0.52690792 ... -0.46905685  0.92502464
   0.78830414]
 [-0.53857819  1.7332077   0.5331366  ... -0.45580254  0.92010193
   0.78570053]
 [-0.53449398  1.73286102  0.53918907 ... -0.44247826  0.91467903
   0.78232964]]
(600, 9) [[ 0.06492162  0.95587555  0.02759707 ... -0.17307722  0.12311673
   0.24024141]
 [ 0.06270361  0.95787239  0.0263968  ... -0.17276836  0.12295028
   0.24042221]
 [ 0.0633704   0.95726311  0.02722497 ... -0.17269129  0.12314113
   0.24044138]
 ...
 [-0.57930702  1.08024368  0.59377594 ... -0.49833176  0.22044437
   0.48325714]
 [-0.57743893  1.08168465  0.59865074 ... -0.49415901  0.22242344
   0.50142422]
 [-0.57473312  1.08314499  0.60363243 ... -0.48933487  0.22346

# Reshape Training Data

In [9]:

def ReshapeData(x, y):
    x_reshaped = np.expand_dims(x, axis=1)
    y_reshaped = np.expand_dims(y, axis=1)

    return x_reshaped, y_reshaped


In [99]:
x_train, y_train = ReshapeData(x_samples, y_samples)

print(x_train.shape, y_train.shape)

print(x_train.shape[1])

print(x_train.shape[2])


(600, 1, 9) (600, 1, 9)
1
9


# Create Test / Validation Data

In [92]:
#Create a single test data file

test_path = "../test_data/"


test_dataframe = ReadCSV(test_path)

#split features and labels into seperate dataframes
x_test_df, y_test_df = SplitFeaturesLabels(test_dataframe)

#convert features and labels to numpy array
x_test, y_test = LoadArray(x_test_df, y_test_df)

#Divide values in array by 100
x_test_normalized, y_test_normalized = NormalizeValues(x_test, y_test)

x_test_samples, y_test_samples = SampleSize(x_test_normalized, y_test_normalized)

x_test, y_test = ReshapeData(x_test_samples, y_test_samples)

print(x_test.shape, y_test.shape)

print(x_test.shape[1])

KeyboardInterrupt: Interrupted by user

# Combine Test and Train datasets

Combine all data sets in /train_data and /test_data into one, for more samples when training 

In [4]:
import os
import pandas as pd
import numpy as np


#Combine all datasets in a directory into one dataframe
def CombineDatasets(path):
    data_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            
            data_list.append(df)
         

    data_df = pd.concat(data_list, axis=0, ignore_index=True)
    return data_df



In [5]:

#combine data in a directory into two lists of x and y features
def DatasetsLists(path):
    x_list = []
    y_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            x_features, y_features = SplitFeaturesLabels(df)
            x_train, y_train = LoadArray(x_features, y_features)
            x_normalized, y_normalized = NormalizeValues(x_train, y_train)
            x_reshape, y_reshape = ReshapeData(x_normalized, y_normalized)
            x_list.append(x_reshape)
            y_list.append(y_reshape)
    return x_list, y_list



In [10]:
train_path = "../train_data/"
x, y = DatasetsLists(train_path)


.ipynb_checkpoints
arm_raise_1_trimmed.csv
arm_raise_3_trimmed.csv
jumping_1_trimmed.csv
jumping_2_trimmed.csv
jumping_3_trimmed.csv
leg_raise_1_trimmed.csv
leg_raise_2_trimmed.csv
leg_raise_3_trimmed.csv
leg_raise_4_trimmed.csv
sitting_standing_1_trimmed.csv
sitting_standing_2_trimmed.csv
sitting_standing_3_trimmed.csv
walking_1_train.csv
walking_2_test.csv
walking_3_trimmed.csv
walking_5_trimmed.csv


Combine all data in the training data directory 

In [178]:
train_path = "../train_data/"

combined_train_dataframe = CombineDatasets(train_path)

x_train, y_train = SplitFeaturesLabels(combined_train_dataframe)
        
x_train_arr, y_train_arr = LoadArray(x_train, y_train)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train_arr, y_train_arr)

x_train, y_train = ReshapeData(x_train_normalized, y_train_normalized)

print(x_train.shape, y_train.shape)


(11908, 1, 9) (11908, 1, 9)


Combine all data in the test data directory

In [15]:
test_path = "../test_data/"

combined_test_dataframe = CombineDatasets(test_path)

x_test, y_test = SplitFeaturesLabels(combined_test_dataframe)
        
x_test_arr, y_test_arr = LoadArray(x_test, y_test)

x_test_normalized, y_test_normalized = NormalizeValues(x_test_arr, y_test_arr)

x_test, y_test = ReshapeData(x_test_normalized, y_test_normalized)

print(x_test.shape)
print(x_test[0:1,:,:])
print(x_test_normalized.shape)


(3000, 1, 9)
[[[ 0.07488835  1.60079437 -0.06116605  0.29804235  0.82701195
   -0.13233584 -0.16509771  0.80914047 -0.10186529]]]
(3000, 9)


# Model Creation and Training

In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import tensorflow as tf
from keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import SGD
from keras.layers.core import Dense, Activation, Dropout


sgd = SGD(learning_rate=0.001, momentum=0.8, decay=0.999, nesterov=False)

tf.keras.backend.set_floatx('float64')

model = Sequential()
model.add(GRU(16, return_sequences=True, input_shape=(1, 9)))
model.add(Dropout(0.2))
model.add(GRU(16, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(9, activation = "linear"))

model.compile(loss='mse', optimizer="adam")

print ('model compiled')

print (model.summary())

model compiled
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 16)             1296      
                                                                 
 dropout (Dropout)           (None, 1, 16)             0         
                                                                 
 gru_1 (GRU)                 (None, 16)                1632      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 9)                 153       
                                                                 
Total params: 3,081
Trainable params: 3,081
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# fit the model to a given set of features (x) and labels (y)    
def FitModel(x, y):
    model.fit(x, y, epochs=1500,batch_size=256)
       

In [12]:
#Fit model to two equally sized lists of x features and y labels 

def FitToList(x, y):
    for i in range(len(x)):
        print(i)
        print(x[i].shape)
        print(y[i].shape)
        model.fit(x[i], y[i], epochs=1500,batch_size=256, verbose=0)
        


In [13]:
FitToList(x, y)

0
(905, 1, 9)
(905, 1, 9)
1
(600, 1, 9)
(600, 1, 9)
2
(916, 1, 9)
(916, 1, 9)
3
(600, 1, 9)
(600, 1, 9)
4
(600, 1, 9)
(600, 1, 9)
5
(934, 1, 9)
(934, 1, 9)
6
(600, 1, 9)
(600, 1, 9)
7
(600, 1, 9)
(600, 1, 9)
8
(600, 1, 9)
(600, 1, 9)
9
(1103, 1, 9)
(1103, 1, 9)
10
(600, 1, 9)
(600, 1, 9)
11
(600, 1, 9)
(600, 1, 9)
12
(972, 1, 9)
(972, 1, 9)
13
(1078, 1, 9)
(1078, 1, 9)
14
(600, 1, 9)
(600, 1, 9)
15
(600, 1, 9)
(600, 1, 9)


In [16]:
model.evaluate(x_test, y_test, batch_size=256)

12/12 [==============================] - 0s 3ms/step - loss: 0.0654


0.06542097796996435

In [19]:
range_1 = 2000
range_2 = 2050

predictions = model.predict(x_test[range_1:range_2,:,:])
arr = y_test.reshape(-1,9)


In [20]:
print("predictions shape:", predictions.shape)

prediction_DF = pd.DataFrame(predictions, columns=["Waist_X", "Waist_Y", "Waist_Z", "Rigth_Foot_X", "Right_Foot_Y", "Right_Foot_Z", "Left_Foot_X", "Left_Foot_Y", "Left_Foot_Z"])
Actual_DF = pd.DataFrame(arr[range_1:range_2], columns=["Waist_X", "Waist_Y", "Waist_Z", "Rigth_Foot_X", "Right_Foot_Y", "Right_Foot_Z", "Left_Foot_X", "Left_Foot_Y", "Left_Foot_Z"])


print("Actual Values")
display(Actual_DF)
print("Predicited Values")
display(prediction_DF)



predictions shape: (50, 9)
Actual Values


,Waist_X,Waist_Y,Waist_Z,Rigth_Foot_X,Right_Foot_Y,Right_Foot_Z,Left_Foot_X,Left_Foot_Y,Left_Foot_Z
0,0.180749,1.018222,-0.078071,0.128831,0.120280,-0.047938,0.001302,0.130428,-0.181753
1,0.181260,1.017639,-0.076536,0.128789,0.120517,-0.046701,0.000489,0.126945,-0.181648
2,0.182131,1.016409,-0.074972,0.129429,0.121071,-0.045191,-0.000736,0.123891,-0.181511
3,0.181732,1.014807,-0.073532,0.130336,0.122684,-0.043273,-0.002018,0.120083,-0.180743
4,0.180403,1.013285,-0.072645,0.130467,0.123244,-0.042119,-0.004495,0.118104,-0.180332
5,0.179394,1.011457,-0.071387,0.131003,0.123943,-0.040986,-0.006415,0.116586,-0.180334
6,0.177270,1.008721,-0.071681,0.131376,0.123901,-0.040497,-0.008176,0.115199,-0.179886
7,0.175114,1.006739,-0.071031,0.131675,0.123948,-0.039978,-0.010239,0.114132,-0.179333
8,0.171516,1.004271,-0.072028,0.131782,0.123980,-0.039895,-0.012438,0.113673,-0.179159
9,0.167185,1.001691,-0.073309,0.132178,0.123892,-0.039678,-0.014404,0.112989,-0.178877


Predicited Values


,Waist_X,Waist_Y,Waist_Z,Rigth_Foot_X,Right_Foot_Y,Right_Foot_Z,Left_Foot_X,Left_Foot_Y,Left_Foot_Z
0,-0.117515,1.017878,0.337680,-0.151143,0.157235,0.392306,-0.137793,0.150065,0.409571
1,-0.117517,1.017877,0.337683,-0.151146,0.157235,0.392311,-0.137795,0.150065,0.409576
2,-0.117520,1.017877,0.337686,-0.151151,0.157235,0.392317,-0.137798,0.150065,0.409581
3,-0.117524,1.017877,0.337689,-0.151158,0.157235,0.392323,-0.137802,0.150065,0.409586
4,-0.117530,1.017877,0.337694,-0.151167,0.157235,0.392330,-0.137808,0.150065,0.409593
5,-0.117536,1.017877,0.337698,-0.151176,0.157235,0.392337,-0.137814,0.150065,0.409599
6,-0.117542,1.017877,0.337702,-0.151186,0.157235,0.392345,-0.137820,0.150065,0.409605
7,-0.117550,1.017877,0.337706,-0.151198,0.157235,0.392352,-0.137828,0.150065,0.409612
8,-0.117558,1.017877,0.337709,-0.151210,0.157235,0.392359,-0.137835,0.150065,0.409618
9,-0.117568,1.017877,0.337714,-0.151224,0.157235,0.392367,-0.137844,0.150065,0.409624
